In [1]:
from neo4j import GraphDatabase
import pandas as pd  
import numpy as np  
%matplotlib inline
import plotly
import plotly as py
import plotly.graph_objs as go
import datetime
py.offline.init_notebook_mode(connected=True)
%config IPCompleter.greedy=True

url = "bolt://localhost:7687"
driver = GraphDatabase.driver(url, auth=("neo4j", "123456"))

# Query 1
query1="Match(m:Movie)return m.title,m.releaseDate,m.genre,m.runtime"
movieYearGenre=[]
with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(query1)
for node in nodes:
    if(type(node["m.releaseDate"])==str):
        hellNo=datetime.datetime.fromtimestamp(int(node["m.releaseDate"])/1000).year
        if(node["m.title"]=='Rodan'):
            movieYearGenre.append([node["m.title"],1956,node["m.genre"],node["m.runtime"]])
        else:
            movieYearGenre.append([node["m.title"],hellNo,node["m.genre"],node["m.runtime"]])
movieYearGenre.sort(key=lambda x: x[1])
year=[]
countMovies=[]
i=movieYearGenre[0][1]
j=0
print(i)
while(i<=movieYearGenre[len(movieYearGenre)-1][1]):
    count=0
    while(j<len(movieYearGenre)and i==movieYearGenre[j][1]):
        count+=1
        j+=1
    year.append(i)
    countMovies.append(count)
    i+=1

print(type(movieYearGenre[0][3]))
layout=go.Layout(
    title="First Query",
    yaxis=dict(title='Count'),
    xaxis=dict(title='Year'))



#Line 

trace1=go.Scatter(
    x=year,
    y=countMovies,
    mode='lines'

)

#Scatter

trace2=go.Scatter(
    x=year,
    y=countMovies,
    mode = 'markers'

)

#Scatter and Lines

trace3=go.Scatter(
    x=year,
    y=countMovies,
    mode='lines+markers'
)
fig=go.Figure(data=[trace3],layout=layout)
py.offline.iplot(fig)


1887
<class 'int'>


In [3]:
#Second Query

query2="MATCH (a:Actor),(m:Movie),(a)-[r:ACTS_IN]->(m) RETURN a.name,count(m) ORDER BY count(m)"
with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(query2)
result=[]
actors=[]
noOfMovies=[]
for node in nodes:
    actors.append(node["a.name"])
    noOfMovies.append(node["count(m)"])



layout5=go.Layout(
    title="Second Query",
    yaxis=dict(title='Total count of Movies'),
    xaxis=dict(title='Actors')
)


trace5=go.Bar(
    x=actors,
    y=noOfMovies,

)
trace = go.Scatter(
    x = actors,
    y = noOfMovies,
    mode='lines'
)



fig=go.Figure(data=[trace],layout=layout5)
py.offline.iplot(fig)





In [8]:
#Query 3

def getListOfActorMovies(actor): 
        q3="Match(a:Actor{name:\""+actor+"\"}),(m:Movie),(a)-[:ACTS_IN]->(m) return m.title,m.releaseDate"
        with driver.session() as graphDB_Session:
            nodes = graphDB_Session.run(q3)
        movies=[]
        for node in nodes:
            movies.append([node["m.title"],datetime.datetime.fromtimestamp(int(node["m.releaseDate"])/1000).year])
        movies.sort(key=lambda x: x[1])
        return movies

query4="MATCH (a:Actor),(m:Movie),(a)-[r:ACTS_IN]->(m) RETURN a.name,count(m) ORDER BY count(m) Desc Limit 10"
with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(query4)
topActors=[]
tMovieNo=[]
countPerY=[]
for node in nodes:
    topActors.append(node["a.name"])
    tMovieNo.append(node["count(m)"])

actorCountYear=[]
for i in topActors:
    moviesofA=getListOfActorMovies(i[0])
    countYear=[]
    for j in year:
        count=0
        for k in moviesofA:
            if k[1]==j:
                count+=1
        countYear.append([count,j])
    actorCountYear.append([i,countYear])

    
        

# layout5=go.Layout(
#     title="Third Query",
#     yaxis=dict(title='Total count and count per year'),
#     xaxis=dict(title='Actors')
# )


# trace5=go.Bar(
#     x=topActors,
#     y=tMovieNoAt,

# )



# fig=go.Figure(data=[trace5],layout=layout5)
# py.offline.iplot(fig)


In [4]:
#Query 4

query5="Match(m:Movie)return Distinct m.genre"
with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(query5)

genres=[]
for node in nodes:
    genres.append(node["m.genre"])
genreYearsCounts=[]
for i in genres:
    j=movieYearGenre[0][1]
    yearsCounts=[]
    k=0
    while j<=movieYearGenre[len(movieYearGenre)-1][1]:
        count=0
        while k<len(movieYearGenre) and j==movieYearGenre[k][1]:
            if movieYearGenre[k][2]==i:
                count+=1
            k+=1
        yearsCounts.append(count)
        j+=1
    genreYearsCounts.append([i,yearsCounts])
print(genres)



layout=go.Layout(
    title="Fouth Query",
    yaxis=dict(title='Count&Geners'),
    xaxis=dict(title='Year'))

trace1 = go.Bar(
    x=year,
    y=genreYearsCounts[0][1],
    name='Film Noir'
)
trace2 = go.Bar(
    x=year,
    y=genreYearsCounts[1][1],
    name='Musical'
)
trace3 = go.Bar(
    x=year,
    y=genreYearsCounts[2][1],
    name='Eastern'
)
trace4 = go.Bar(
    x=year,
    y=genreYearsCounts[3][1],
    name='History'
)
trace5 = go.Bar(
    x=year,
    y=genreYearsCounts[4][1],
    name='Thriller'
)
trace6 = go.Bar(
    x=year,
    y=genreYearsCounts[5][1],
    name='short'
)
trace7 = go.Bar(
    x=year,
    y=genreYearsCounts[6][1],
    name='Music'
)
trace8 = go.Bar(
    x=year,
    y=genreYearsCounts[7][1],
    name='Suspense'
)
trace9 = go.Bar(
    x=year,
    y=genreYearsCounts[8][1],
    name='Drama'
)
trace10 = go.Bar(
    x=year,
    y=genreYearsCounts[9][1],
    name='Disaster'
)
trace11 = go.Bar(
    x=year,
    y=genreYearsCounts[10][1],
    name='Western'
)
trace12 = go.Bar(
    x=year,
    y=genreYearsCounts[11][1],
    name='Erotic'
)
trace13 = go.Bar(
    x=year,
    y=genreYearsCounts[12][1],
    name='Adventure'
)
trace14 = go.Bar(
    x=year,
    y=genreYearsCounts[13][1],
    name='Road Movie'
)
trace15 = go.Bar(
    x=year,
    y=genreYearsCounts[14][1],
    name='Indie'
)
trace16 = go.Bar(
    x=year,
    y=genreYearsCounts[15][1],
    name='Mystery'
)
trace17 = go.Bar(
    x=year,
    y=genreYearsCounts[16][1],
    name='Science Fiction'
)
trace18 = go.Bar(
    x=year,
    y=genreYearsCounts[17][1],
    name='Horror'
)
trace19 = go.Bar(
    x=year,
    y=genreYearsCounts[18][1],
    name='British'
)
trace20 = go.Bar(
    x=year,
    y=genreYearsCounts[19][1],
    name='Documentry'
)
trace21 = go.Bar(
    x=year,
    y=genreYearsCounts[20][1],
    name='None'
)
trace22 = go.Bar(
    x=year,
    y=genreYearsCounts[21][1],
    name='Animation'
)

trace23 = go.Bar(
    x=year,
    y=genreYearsCounts[22][1],
    name='Family'
)
trace24 = go.Bar(
    x=year,
    y=genreYearsCounts[23][1],
    name='War'
)
trace25 = go.Bar(
    x=year,
    y=genreYearsCounts[24][1],
    name='Sporting Event'
)
trace26 = go.Bar(
    x=year,
    y=genreYearsCounts[25][1],
    name='Romance'
)
trace27 = go.Bar(
    x=year,
    y=genreYearsCounts[26][1],
    name='Crime'
)
trace28 = go.Bar(
    x=year,
    y=genreYearsCounts[27][1],
    name='Action'
)
trace29 = go.Bar(
    x=year,
    y=genreYearsCounts[28][1],
    name='Comedy'
)
trace30 = go.Bar(
    x=year,
    y=genreYearsCounts[29][1],
    name='Fantasy'
)
trace31 = go.Bar(
    x=year,
    y=genreYearsCounts[30][1],
    name='Sports Film'
)

data = [trace1, trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10,trace11,trace12,trace13,trace14,trace15,trace16,trace17,trace18,trace20,trace21,trace22,trace23,trace24,trace25,trace26,trace27,trace28,trace29,trace30,trace31]
layout = go.Layout(
    barmode='stack'
)


fig=go.Figure(data=data,layout=layout)
py.offline.iplot(fig)








['Film Noir', 'Musical', 'Eastern', 'History', 'Thriller', 'Short', 'Music', 'Suspense', 'Drama', 'Disaster', 'Western', 'Erotic', 'Adventure', 'Road Movie', 'Indie', 'Mystery', 'Science Fiction', 'Horror', 'British', 'Documentary', 'Animation', 'Family', 'War', 'Sporting Event', None, 'Romance', 'Crime', 'Action', 'Comedy', 'Fantasy', 'Sports Film']


In [5]:
#Query 5
avTimeOverYear=[]
j=movieYearGenre[0][1]
k=0
while j<=movieYearGenre[len(movieYearGenre)-1][1]:
    sum=0
    count=0
    while k<len(movieYearGenre) and j==movieYearGenre[k][1]:
        count+=1
        if(type(movieYearGenre[k][3])==int and movieYearGenre[k][3]>0):
            sum+=movieYearGenre[k][3]
        k+=1
    if(count==0):
        count=1
    avTimeOverYear.append(sum/count)  
    j+=1
    

layout=go.Layout(
    title="Fifth Query",
    yaxis=dict(title='Average running time'),
    xaxis=dict(title='Year')) 
trace = go.Scatter(
    x=year,
    y=avTimeOverYear,
)
fig=go.Figure(data=[trace],layout=layout)
py.offline.iplot(fig)



In [15]:
#Query 6
avTimeOverYearGs=[]
for i in genres:
    avTimeOverYearG=[]
    k=0
    j=movieYearGenre[0][1]
    while j<=movieYearGenre[len(movieYearGenre)-1][1]:
        sum=0
        count=0
        while k<len(movieYearGenre) and j==movieYearGenre[k][1]:
            if movieYearGenre[k][2]==i:
                count+=1
                if(type(movieYearGenre[k][3])==int and movieYearGenre[k][3]>0):
                    sum+=movieYearGenre[k][3]
            k+=1
        if(count==0):
            count=1
        avTimeOverYearG.append(sum/count)  
        j+=1
    avTimeOverYearGs.append([i,avTimeOverYearG])

layout=go.Layout(
    title="Query 6",
    yaxis=dict(title='avgRunTime&Geners'),
    xaxis=dict(title='Year'))

trace1 = go.Bar(
    x=year,
    y=avTimeOverYearGs[0][1],
    name='Film Noir'
)
trace2 = go.Bar(
    x=year,
    y=avTimeOverYearGs[1][1],
    name='Musical'
)
trace3 = go.Bar(
    x=year,
    y=avTimeOverYearGs[2][1],
    name='Eastern'
)
trace4 = go.Bar(
    x=year,
    y=avTimeOverYearGs[3][1],
    name='History'
)
trace5 = go.Bar(
    x=year,
    y=avTimeOverYearGs[4][1],
    name='Thriller'
)
trace6 = go.Bar(
    x=year,
    y=avTimeOverYearGs[5][1],
    name='short'
)
trace7 = go.Bar(
    x=year,
    y=avTimeOverYearGs[6][1],
    name='Music'
)
trace8 = go.Bar(
    x=year,
    y=avTimeOverYearGs[7][1],
    name='Suspense'
)
trace9 = go.Bar(
    x=year,
    y=avTimeOverYearGs[8][1],
    name='Drama'
)
trace10 = go.Bar(
    x=year,
    y=avTimeOverYearGs[9][1],
    name='Disaster'
)
trace11 = go.Bar(
    x=year,
    y=avTimeOverYearGs[10][1],
    name='Western'
)
trace12 = go.Bar(
    x=year,
    y=avTimeOverYearGs[11][1],
    name='Erotic'
)
trace13 = go.Bar(
    x=year,
    y=avTimeOverYearGs[12][1],
    name='Adventure'
)
trace14 = go.Bar(
    x=year,
    y=avTimeOverYearGs[13][1],
    name='Road Movie'
)
trace15 = go.Bar(
    x=year,
    y=avTimeOverYearGs[14][1],
    name='Indie'
)
trace16 = go.Bar(
    x=year,
    y=avTimeOverYearGs[15][1],
    name='Mystery'
)
trace17 = go.Bar(
    x=year,
    y=avTimeOverYearGs[16][1],
    name='Science Fiction'
)
trace18 = go.Bar(
    x=year,
    y=avTimeOverYearGs[17][1],
    name='Horror'
)
trace19 = go.Bar(
    x=year,
    y=avTimeOverYearGs[18][1],
    name='British'
)
trace20 = go.Bar(
    x=year,
    y=avTimeOverYearGs[19][1],
    name='Documentry'
)
trace21 = go.Bar(
    x=year,
    y=avTimeOverYearGs[20][1],
    name='None'
)
trace22 = go.Bar(
    x=year,
    y=avTimeOverYearGs[21][1],
    name='Animation'
)

trace23 = go.Bar(
    x=year,
    y=avTimeOverYearGs[22][1],
    name='Family'
)
trace24 = go.Bar(
    x=year,
    y=avTimeOverYearGs[23][1],
    name='War'
)
trace25 = go.Bar(
    x=year,
    y=avTimeOverYearGs[24][1],
    name='Sporting Event'
)
trace26 = go.Bar(
    x=year,
    y=avTimeOverYearGs[25][1],
    name='Romance'
)
trace27 = go.Bar(
    x=year,
    y=avTimeOverYearGs[26][1],
    name='Crime'
)
trace28 = go.Bar(
    x=year,
    y=avTimeOverYearGs[27][1],
    name='Action'
)
trace29 = go.Bar(
    x=year,
    y=avTimeOverYearGs[28][1],
    name='Comedy'
)
trace30 = go.Bar(
    x=year,
    y=avTimeOverYearGs[29][1],
    name='Fantasy'
)
trace31 = go.Bar(
    x=year,
    y=avTimeOverYearGs[30][1],
    name='Sports Film'
)

data = [trace1, trace2,trace3,trace4,trace5,trace6,trace7,trace8,trace9,trace10,trace11,trace12,trace13,trace14,trace15,trace16,trace17,trace18,trace20,trace21,trace22,trace23,trace24,trace25,trace26,trace27,trace28,trace29,trace30,trace31]
layout = go.Layout(
    barmode='stack'
)


fig=go.Figure(data=data,layout=layout)
py.offline.iplot(fig)



In [12]:
#Query 7

rygs=[]
def getGRCount(movie): 
        q6="MATCH (m:Movie{title:\""+movie+"\"})<-[r:RATED]-() RETURN  count(r)"
        count=0
        with driver.session() as graphDB_Session:
            nodes = graphDB_Session.run(q6)
        for node in nodes:
            count+=node["count(r)"]
        return count

for i in genres:
    j=movieYearGenre[0][1]
    k=0
    ryg=[]
    while j<=movieYearGenre[len(movieYearGenre)-1][1]:
        count =0
        while k< len(movieYearGenre)and movieYearGenre[k][1]==j:
            if(movieYearGenre[k][2]==i):
                count+=getGRCount(movieYearGenre[k][0])             
            k+=1
            ryg.append(count)
        j+=1
        rygs.append([i,ryg])
        
                
    


In [35]:
def changeTimeStampToDate(node):
    if(type(node["m.releaseDate"])==str):
        hellNo=datetime.datetime.fromtimestamp(int(node["m.releaseDate"])/1000).year
        if(node["m.title"]=='Rodan'):
            return 1956
        else:
            return hellNo
            


def getGRCount(movie): 
        q6="MATCH (m:Movie{title:\""+movie+"\"})<-[r:RATED]-() RETURN  count(r)"
        count=0
        with driver.session() as graphDB_Session:
            nodes = graphDB_Session.run(q6)
            for node in nodes:
                count+=node["count(r)"]
        return count
    
def getMoviesWithRates():
    moviesNYGWithRates=[]
    gens=[]
    yearsWRates=[]
    q7="MATCH (m:Movie)<-[r:RATED]-() RETURN  Distinct m.title,m.genre,m.releaseDate"
    with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(q7)
    for node in nodes:
        moviesNYGWithRates.append([node["m.title"],node["m.genre"],changeTimeStampToDate(node)])
        gens.append(node["m.genre"])
        if(not(changeTimeStampToDate(node) in yearsWRates)):
            yearsWRates.append(changeTimeStampToDate(node))
    return moviesNYGWithRates,list(dict.fromkeys(gens)),yearsWRates

moviesWithRates,gens,yearsWRates=getMoviesWithRates()
moviesWithRates.sort(key=lambda x: x[2])
yearsWRates.sort() 
print(yearsWRates)


movieRPGY=[]
for i in gens:
    j=moviesWithRates[0][2]
    k=0
    countPY=[]
    for j in yearsWRates:
        count=0
        while k<len(moviesWithRates) and moviesWithRates[k][2]==j:
            if i==moviesWithRates[k][1]:
                count+=getGRCount(moviesWithRates[k][0])
            k+=1
        countPY.append(count)
    movieRPGY.append([i,countPY])
    
    
layout=go.Layout(
    title="7th Query",
    yaxis=dict(title='Count of Rates'),
    xaxis=dict(title='Year'),
    barmode='stack',
    
) 

trace71 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[0][1],
   name='Action'
)
trace72 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[1][1],
   name='Drama'
)
trace73 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[2][1],
   name='Animation'
)
trace74 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[3][1],
   name='Crime'
)
trace75 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[4][1],
   name='Comedy'
)
trace76 = go.Bar(
   x=yearsWRates,
   y=movieRPGY[5][1],
   name='None'
)
fig=go.Figure(data=[trace71,trace72,trace73,trace74,trace75,trace76],layout=layout)
py.offline.iplot(fig)


    
    
    
    
    
    
    
    
    

[1939, 1958, 1979, 1985, 1992, 1994, 1997, 1999, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]


In [22]:
count=0
print(len(movieYearGenre))
for i in range (len(movieYearGenre)):
    print("Hi->"+str(i))
    q6="MATCH (m:Movie{title:\""+movieYearGenre[i][0]+"\"})<-[r:RATED]-() RETURN  count(r)"
    with driver.session() as graphDB_Session:
        nodes = graphDB_Session.run(q6)
    for node in nodes:
        count+=node["count(r)"]
    print(count)
        
        
    
    

12613
Hi->0
0
Hi->1
0
Hi->2
0
Hi->3
0
Hi->4
0
Hi->5
0
Hi->6
0
Hi->7
0
Hi->8
0
Hi->9
0
Hi->10
0
Hi->11
0
Hi->12
0
Hi->13
0
Hi->14
0
Hi->15
0
Hi->16
0
Hi->17
0
Hi->18
0
Hi->19
0
Hi->20
0
Hi->21
0
Hi->22
0
Hi->23
0
Hi->24
0
Hi->25
0
Hi->26
0
Hi->27
0
Hi->28
0
Hi->29
0
Hi->30
0
Hi->31
0
Hi->32
0
Hi->33
0
Hi->34
0
Hi->35
0
Hi->36
0
Hi->37
0
Hi->38
0
Hi->39
0
Hi->40
0
Hi->41
0
Hi->42
0
Hi->43
0
Hi->44
0
Hi->45
0
Hi->46
0
Hi->47
0
Hi->48
0
Hi->49
0
Hi->50
0
Hi->51
0
Hi->52
0
Hi->53
0
Hi->54
0
Hi->55
0
Hi->56
0
Hi->57
0
Hi->58
0
Hi->59
0
Hi->60
0
Hi->61
0
Hi->62
0
Hi->63
0
Hi->64
0
Hi->65
0
Hi->66
0
Hi->67
0
Hi->68
0
Hi->69
0
Hi->70
0
Hi->71
0
Hi->72
0
Hi->73
0
Hi->74
0
Hi->75
0
Hi->76
0
Hi->77
0
Hi->78
0
Hi->79
0
Hi->80
0
Hi->81
0
Hi->82
0
Hi->83
0
Hi->84
0
Hi->85
0
Hi->86
0
Hi->87
0
Hi->88
0
Hi->89
0
Hi->90
0
Hi->91
0
Hi->92
0
Hi->93
0
Hi->94
0
Hi->95
0
Hi->96
0
Hi->97
0
Hi->98
0
Hi->99
0
Hi->100
0
Hi->101
0
Hi->102
0
Hi->103
0
Hi->104
0
Hi->105
0
Hi->106
0
Hi->107
0
Hi->108
0
Hi->109
0
Hi->

1
Hi->837
1
Hi->838
1
Hi->839
1
Hi->840
1
Hi->841
1
Hi->842
1
Hi->843
1
Hi->844
1
Hi->845
1
Hi->846
1
Hi->847
1
Hi->848
1
Hi->849
1
Hi->850
1
Hi->851
1
Hi->852
1
Hi->853
1
Hi->854
1
Hi->855
1
Hi->856
1
Hi->857
1
Hi->858
1
Hi->859
1
Hi->860
1
Hi->861
1
Hi->862
1
Hi->863
1
Hi->864
1
Hi->865
1
Hi->866
1
Hi->867
1
Hi->868
1
Hi->869
1
Hi->870
1
Hi->871
1
Hi->872
1
Hi->873
1
Hi->874
1
Hi->875
1
Hi->876
1
Hi->877
1
Hi->878
1
Hi->879
1
Hi->880
1
Hi->881
1
Hi->882
1
Hi->883
1
Hi->884
1
Hi->885
1
Hi->886
1
Hi->887
1
Hi->888
1
Hi->889
1
Hi->890
1
Hi->891
1
Hi->892
1
Hi->893
1
Hi->894
1
Hi->895
1
Hi->896
1
Hi->897
1
Hi->898
1
Hi->899
1
Hi->900
1
Hi->901
1
Hi->902
1
Hi->903
1
Hi->904
1
Hi->905
1
Hi->906
1
Hi->907
1
Hi->908
1
Hi->909
1
Hi->910
1
Hi->911
1
Hi->912
1
Hi->913
1
Hi->914
1
Hi->915
1
Hi->916
1
Hi->917
1
Hi->918
1
Hi->919
1
Hi->920
1
Hi->921
1
Hi->922
1
Hi->923
1
Hi->924
1
Hi->925
1
Hi->926
1
Hi->927
1
Hi->928
1
Hi->929
1
Hi->930
1
Hi->931
1
Hi->932
1
Hi->933
1
Hi->934
1
Hi->935
1
Hi->936


2
Hi->1610
2
Hi->1611
2
Hi->1612
2
Hi->1613
2
Hi->1614
2
Hi->1615
2
Hi->1616
2
Hi->1617
2
Hi->1618
2
Hi->1619
2
Hi->1620
2
Hi->1621
2
Hi->1622
2
Hi->1623
2
Hi->1624
2
Hi->1625
2
Hi->1626
2
Hi->1627
2
Hi->1628
2
Hi->1629
2
Hi->1630
2
Hi->1631
2
Hi->1632
2
Hi->1633
2
Hi->1634
2
Hi->1635
2
Hi->1636
2
Hi->1637
2
Hi->1638
2
Hi->1639
2
Hi->1640
2
Hi->1641
2
Hi->1642
2
Hi->1643
2
Hi->1644
2
Hi->1645
2
Hi->1646
2
Hi->1647
2
Hi->1648
2
Hi->1649
2
Hi->1650
2
Hi->1651
2
Hi->1652
2
Hi->1653
2
Hi->1654
2
Hi->1655
2
Hi->1656
2
Hi->1657
2
Hi->1658
2
Hi->1659
2
Hi->1660
2
Hi->1661
2
Hi->1662
2
Hi->1663
2
Hi->1664
2
Hi->1665
2
Hi->1666
2
Hi->1667
2
Hi->1668
2
Hi->1669
2
Hi->1670
2
Hi->1671
2
Hi->1672
2
Hi->1673
2
Hi->1674
2
Hi->1675
2
Hi->1676
2
Hi->1677
2
Hi->1678
2
Hi->1679
2
Hi->1680
2
Hi->1681
2
Hi->1682
2
Hi->1683
2
Hi->1684
2
Hi->1685
2
Hi->1686
2
Hi->1687
2
Hi->1688
2
Hi->1689
2
Hi->1690
2
Hi->1691
2
Hi->1692
2
Hi->1693
2
Hi->1694
2
Hi->1695
2
Hi->1696
2
Hi->1697
2
Hi->1698
2
Hi->1699
2
Hi->1700

3
Hi->2356
3
Hi->2357
3
Hi->2358
3
Hi->2359
3
Hi->2360
3
Hi->2361
3
Hi->2362
3
Hi->2363
3
Hi->2364
3
Hi->2365
3
Hi->2366
3
Hi->2367
3
Hi->2368
3
Hi->2369
3
Hi->2370
3
Hi->2371
3
Hi->2372
3
Hi->2373
3
Hi->2374
3
Hi->2375
3
Hi->2376
3
Hi->2377
3
Hi->2378
3
Hi->2379
3
Hi->2380
3
Hi->2381
3
Hi->2382
3
Hi->2383
3
Hi->2384
3
Hi->2385
3
Hi->2386
3
Hi->2387
3
Hi->2388
3
Hi->2389
3
Hi->2390
3
Hi->2391
3
Hi->2392
3
Hi->2393
3
Hi->2394
3
Hi->2395
3
Hi->2396
3
Hi->2397
3
Hi->2398
3
Hi->2399
3
Hi->2400
3
Hi->2401
3
Hi->2402
3
Hi->2403
3
Hi->2404
3
Hi->2405
3
Hi->2406
3
Hi->2407
3
Hi->2408
3
Hi->2409
3
Hi->2410
3
Hi->2411
3
Hi->2412
3
Hi->2413
3
Hi->2414
3
Hi->2415
3
Hi->2416
3
Hi->2417
3
Hi->2418
3
Hi->2419
3
Hi->2420
3
Hi->2421
3
Hi->2422
3
Hi->2423
3
Hi->2424
3
Hi->2425
3
Hi->2426
3
Hi->2427
3
Hi->2428
3
Hi->2429
3
Hi->2430
3
Hi->2431
3
Hi->2432
3
Hi->2433
3
Hi->2434
3
Hi->2435
3
Hi->2436
3
Hi->2437
3
Hi->2438
3
Hi->2439
3
Hi->2440
3
Hi->2441
3
Hi->2442
3
Hi->2443
3
Hi->2444
3
Hi->2445
3
Hi->2446

4
Hi->3105
4
Hi->3106
4
Hi->3107
4
Hi->3108
4
Hi->3109
4
Hi->3110
4
Hi->3111
4
Hi->3112
4
Hi->3113
4
Hi->3114
4
Hi->3115
4
Hi->3116
4
Hi->3117
4
Hi->3118
4
Hi->3119
4
Hi->3120
4
Hi->3121
4
Hi->3122
4
Hi->3123
4
Hi->3124
4
Hi->3125
4
Hi->3126
4
Hi->3127
4
Hi->3128
4
Hi->3129
4
Hi->3130
4
Hi->3131
4
Hi->3132
4
Hi->3133
4
Hi->3134
4
Hi->3135
4
Hi->3136
4
Hi->3137
4
Hi->3138
4
Hi->3139
4
Hi->3140
4
Hi->3141
4
Hi->3142
4
Hi->3143
4
Hi->3144
4
Hi->3145
4
Hi->3146
4
Hi->3147
4
Hi->3148
4
Hi->3149
4
Hi->3150
4
Hi->3151
4
Hi->3152
4
Hi->3153
4
Hi->3154
4
Hi->3155
4
Hi->3156
4
Hi->3157
4
Hi->3158
4
Hi->3159
4
Hi->3160
4
Hi->3161
4
Hi->3162
4
Hi->3163
4
Hi->3164
4
Hi->3165
4
Hi->3166
4
Hi->3167
4
Hi->3168
4
Hi->3169
4
Hi->3170
4
Hi->3171
4
Hi->3172
4
Hi->3173
4
Hi->3174
4
Hi->3175
4
Hi->3176
4
Hi->3177
4
Hi->3178
4
Hi->3179
4
Hi->3180
4
Hi->3181
4
Hi->3182
4
Hi->3183
4
Hi->3184
4
Hi->3185
4
Hi->3186
4
Hi->3187
4
Hi->3188
4
Hi->3189
4
Hi->3190
4
Hi->3191
4
Hi->3192
4
Hi->3193
4
Hi->3194
4
Hi->3195

5
Hi->3856
5
Hi->3857
5
Hi->3858
5
Hi->3859
5
Hi->3860
5
Hi->3861
5
Hi->3862
5
Hi->3863
5
Hi->3864
5
Hi->3865
5
Hi->3866
5
Hi->3867
5
Hi->3868
5
Hi->3869
5
Hi->3870
5
Hi->3871
5
Hi->3872
5
Hi->3873
5
Hi->3874
5
Hi->3875
5
Hi->3876
5
Hi->3877
5
Hi->3878
5
Hi->3879
5
Hi->3880
5
Hi->3881
5
Hi->3882
5
Hi->3883
5
Hi->3884
5
Hi->3885
5
Hi->3886
5
Hi->3887
5
Hi->3888
5
Hi->3889
5
Hi->3890
5
Hi->3891
5
Hi->3892
5
Hi->3893
5
Hi->3894
5
Hi->3895
5
Hi->3896
5
Hi->3897
5
Hi->3898
5
Hi->3899
5
Hi->3900
5
Hi->3901
5
Hi->3902
5
Hi->3903
5
Hi->3904
5
Hi->3905
5
Hi->3906
5
Hi->3907
5
Hi->3908
5
Hi->3909
5
Hi->3910
5
Hi->3911
5
Hi->3912
5
Hi->3913
5
Hi->3914
5
Hi->3915
5
Hi->3916
5
Hi->3917
5
Hi->3918
5
Hi->3919
5
Hi->3920
5
Hi->3921
5
Hi->3922
5
Hi->3923
5
Hi->3924
5
Hi->3925
5
Hi->3926
5
Hi->3927
5
Hi->3928
5
Hi->3929
5
Hi->3930
5
Hi->3931
5
Hi->3932
5
Hi->3933
5
Hi->3934
5
Hi->3935
5
Hi->3936
5
Hi->3937
5
Hi->3938
5
Hi->3939
5
Hi->3940
5
Hi->3941
5
Hi->3942
5
Hi->3943
5
Hi->3944
5
Hi->3945
5
Hi->3946

5
Hi->4609
5
Hi->4610
5
Hi->4611
5
Hi->4612
5
Hi->4613
5
Hi->4614
5
Hi->4615
5
Hi->4616
5
Hi->4617
5
Hi->4618
5
Hi->4619
5
Hi->4620
5
Hi->4621
5
Hi->4622
5
Hi->4623
5
Hi->4624
5
Hi->4625
5
Hi->4626
5
Hi->4627
5
Hi->4628
5
Hi->4629
5
Hi->4630
5
Hi->4631
5
Hi->4632
5
Hi->4633
5
Hi->4634
6
Hi->4635
6
Hi->4636
6
Hi->4637
6
Hi->4638
6
Hi->4639
6
Hi->4640
6
Hi->4641
6
Hi->4642
6
Hi->4643
6
Hi->4644
6
Hi->4645
6
Hi->4646
6
Hi->4647
6
Hi->4648
6
Hi->4649
6
Hi->4650
6
Hi->4651
6
Hi->4652
6
Hi->4653
6
Hi->4654
6
Hi->4655
6
Hi->4656
6
Hi->4657
6
Hi->4658
6
Hi->4659
6
Hi->4660
6
Hi->4661
6
Hi->4662
6
Hi->4663
6
Hi->4664
6
Hi->4665
6
Hi->4666
6
Hi->4667
6
Hi->4668
6
Hi->4669
6
Hi->4670
6
Hi->4671
6
Hi->4672
6
Hi->4673
6
Hi->4674
6
Hi->4675
6
Hi->4676
6
Hi->4677
6
Hi->4678
6
Hi->4679
6
Hi->4680
6
Hi->4681
6
Hi->4682
6
Hi->4683
6
Hi->4684
6
Hi->4685
6
Hi->4686
6
Hi->4687
6
Hi->4688
6
Hi->4689
6
Hi->4690
6
Hi->4691
6
Hi->4692
6
Hi->4693
6
Hi->4694
6
Hi->4695
6
Hi->4696
6
Hi->4697
6
Hi->4698
6
Hi->4699

KeyboardInterrupt: 

In [87]:
readable = datetime.datetime.fromtimestamp(882486000000/1000).year
print(type(readable))

<class 'int'>
